In [13]:
# Young Lee
# Group 2

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import datetime

class MainWindow(tk.Tk):
    def __init__(self):  
        super().__init__() 
        self.geometry("800x600")  # Adjust window size
        self.title('Notebook') 
        self.config(bg='light blue')
        self.notebook = []
        self.notes = [] 
        
        # Customizing styles
        self.style = ttk.Style()
        self.style.configure('Custom.TButton', foreground='black')
        
        # Title entry
        ttk.Label(self, text="Preview title:").grid(row=0, column=0, padx=10, pady=5)
        self.note_title = ttk.Entry(self, width=50)
        self.note_title.grid(row=0, column=1, padx=10, pady=5)
        
        # Notes text area
        ttk.Label(self, text="Notes:").grid(row=1, column=0, padx=10, pady=5)
        self.note_text = tk.Text(self, width=60, height=20)
        self.note_text.grid(row=1, column=1, padx=10, pady=5)
        
        # Main buttons
        ttk.Button(self, text="New", command=self.new_note, style='Custom.TButton').grid(row=2, column=0, padx=5, pady=5, sticky="ew")
        ttk.Button(self, text="Open", command=self.open_notebook, style='Custom.TButton').grid(row=2, column=1, padx=5, pady=5, sticky="ew")
        ttk.Button(self, text="Save", command=self.save_notebook, style='Custom.TButton').grid(row=2, column=2, padx=5, pady=5, sticky="ew")
        ttk.Button(self, text="Quit", command=self.quit_app, style='Custom.TButton').grid(row=2, column=3, padx=5, pady=5, sticky="ew")
        
        # Frame for note buttons
        self.note_list_frame = tk.Frame(self)
        self.note_list_frame.grid(row=3, column=0, columnspan=4, padx=10, pady=10, sticky="nsew")
        self.note_list_frame.grid_rowconfigure(0, weight=1)
        self.note_list_frame.grid_columnconfigure(0, weight=1)
        self.note_buttons = []

    def new_note(self, event=None):
        self.config(bg='light blue')  # Change background color
        note_window = NoteForm(self, self.notebook, self.update_note_list)

    def open_notebook(self):
        filepath = filedialog.askopenfilename(initialdir=".",
                                               filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filepath:
            with open(filepath, "r") as file:
                for line in file:
                    title = line.strip()
                    text = file.readline().strip()
                    meta = file.readline().strip()
                    self.notebook.append(MakeNote({"title": title, "text": text, "meta": meta}))
                self.update_note_list()

    def save_notebook(self):
        filepath = filedialog.asksaveasfilename(defaultextension=".txt",
                                                filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filepath:
            with open(filepath, "w") as file:
                for note in self.notebook:
                    file.write(f"{note.title}\n{note.text}\n{note.meta}\n")
                    
    def update_note_list(self):
        for button in self.note_buttons:
            button.destroy()
        self.note_buttons.clear()

        for index, note in enumerate(self.notebook):
            button = ttk.Button(self.note_list_frame, text=note.title, style='Custom.TButton',
                                command=lambda i=index: self.display(i))
            button.grid(row=index, column=0, padx=5, pady=5, sticky="ew")
            self.note_buttons.append(button)

    def display(self, index):
        selected_note = self.notebook[index]
        popup = tk.Toplevel()
        popup.title(selected_note.title)
        popup.geometry("400x300")  # Adjust pop-up window size
        
        # Display note details using labels
        ttk.Label(popup, text=f"Title: {selected_note.title}").pack(anchor="w", padx=10, pady=5)
        ttk.Label(popup, text=f"Text:").pack(anchor="w", padx=10, pady=5)
        text_scroll = tk.Scrollbar(popup, orient="vertical")
        text_scroll.pack(side="right", fill="y")
        text_widget = tk.Text(popup, wrap="word", yscrollcommand=text_scroll.set)
        text_widget.insert("1.0", selected_note.text)
        text_widget.pack(fill="both", expand=True)
        text_scroll.config(command=text_widget.yview)

    def quit_app(self):
        self.quit()  # Close the application

class NoteForm(tk.Toplevel):
    def __init__(self, master, notebook, update_note_list):
        super().__init__(master)
        self.update_note_list = update_note_list

        # Note form elements
        ttk.Label(self, text="Title:").grid(row=0, column=0, padx=10, pady=5)
        self.note_title = ttk.Entry(self, width=50)
        self.note_title.grid(row=0, column=1, padx=10, pady=5)

        ttk.Label(self, text="Text:").grid(row=1, column=0, padx=10, pady=5)
        self.note_text = tk.Text(self, width=60, height=10)
        self.note_text.grid(row=1, column=1, padx=10, pady=5)

        ttk.Button(self, text="Submit", command=self.submit, style='Custom.TButton').grid(row=2, column=0, columnspan=2, padx=10, pady=5)
        
        # Additional note fields
        ttk.Label(self, text="Links:").grid(row=3, column=0, padx=10, pady=5)
        self.note_link = tk.Entry(self, width=50)
        self.note_link.grid(row=3, column=1, padx=10, pady=5)
        self.note_link.insert(0, 'If there is a link with this note enter it here. Else, enter "None"') 

        ttk.Label(self, text="Tags:").grid(row=4, column=0, padx=10, pady=5)
        self.note_tags = tk.Entry(self, width=50)
        self.note_tags.grid(row=4, column=1, padx=10, pady=5)
        self.note_tags.insert(0, 'Enter #tags, separated by commas') 

        self.notebook = notebook

    def submit(self):
        created = datetime.datetime.now()
        local_now = created.astimezone()
        local_tz = local_now.tzinfo 
        title = self.note_title.get()
        text = self.note_text.get("1.0", tk.END).strip()
        meta = f'created {created}, {local_tz}'
        new_note = MakeNote({"title": title, "text": text, "meta": meta})
        self.notebook.append(new_note)
        self.update_note_list()
        self.destroy()

class MakeNote:
    def __init__(self, note_dict):
        self.title = note_dict["title"]
        self.text = note_dict["text"]
        self.meta = note_dict["meta"]

if __name__ == '__main__':
    main_window = MainWindow() 
    main_window.mainloop()


In [4]:
# print your notes here
import os
import json 

downloads_folder = os.path.expanduser("~/Downloads")
json_file_path = os.path.join(downloads_folder, "3notes_for_inst326.json")
hand = open(json_file_path)
for pen in hand:
    print(pen)

[

    {

        "title": "Versatility and Application",

        "text": "Python's versatility makes it a valuable tool for college students across various disciplines. From data analysis in STEM fields to web development and automation in business or humanities, Python's wide range of libraries and frameworks cater to diverse needs. Learning Python equips students with a practical skillset applicable in academia and beyond.",

        "link": "link",

        "tags": "python, versatility, application",

        "date_created": "2024-03-31 17:49:30"

    },

    {

        "title": "Community and Resources",

        "text": "Python boasts a vibrant community with extensive documentation, tutorials, and forums. College students can leverage this wealth of resources to deepen their understanding, troubleshoot issues, and collaborate on projects. Platforms like Stack Overflow, GitHub, and Python-specific forums offer invaluable support for beginners and advanced learners alike.",

    